In [ ]:
# ALGORITHM 1

%run utilities.ipynb

def _algorithm1(gens, l, k, collect_data=False):
    n = l**k
    R = _R(n)
    G = GL(2, R).subgroup(gens)

    def _as_pair(v):
        return (int(v[0]), int(v[1]))
    
    def num_exact_order_vectors_lk(l, k):
        if k == 0:
            return 0
        n = l**k
        return n*n - (n//l)*(n//l)   # l^(2k) - l^(2k-2)
    
    def orbit_size_on_vectors(G, v0):
        # Use cached orbit computation if available
        gens_list = list(G.gens())
        gen_key = tuple(_mat_key(g) for g in gens_list)
        v_key = _as_pair(v0)
        cache_key = (gen_key, v_key, l**k)
        
        if cache_key in _orbit_cache:
            return _orbit_cache[cache_key]
        
        gens_with_inv = gens_list + [g.inverse() for g in gens_list]
        
        seen = set()
        stack = [v0]
        seen.add(v_key)
        
        while stack:
            v = stack.pop()
            for g in gens_with_inv:
                w = g * v
                key = _as_pair(w)
                if key not in seen:
                    seen.add(key)
                    stack.append(w)
        
        result = len(seen)
        _orbit_cache[cache_key] = result
        return result
    
    # Early exit: check if orbit size matches expected
    n = l**k
    v = vector(R, [1, 0])  # exact order n for all l,k>=1
    orb_size = orbit_size_on_vectors(G, v)
    expected_size = num_exact_order_vectors_lk(l, k)
    
    algo1_result = orb_size == expected_size
    
    if collect_data:
        # Return detailed data for JSONL output
        target_size = n*n - (n//l)*(n//l)  # |V_{l^k}|
        lt_status = "PASS" if algo1_result else "FAIL"
        lt_deficit = 0 if algo1_result else target_size
        
        return {
            "result": algo1_result,
            "status": lt_status,
            "orbit_size": int(orb_size),
            "target_size": int(target_size),
            "deficit": int(lt_deficit)
        }
    
    return algo1_result